# Чтение

In [39]:
# MAX_ID = 20000 for poss_set_small_2_dist.txt
# MAX_ID = 10000 for poss_set_small.txt
# MAX_ID = -1 for poss_set.txt
MAX_ID = -1

In [50]:
fin_train = open('train.in', 'r')
train_set = set()
nodes = set()
for line in fin_train:
    line = line.strip()
    u, w = line.split()
    if MAX_ID == -1 or (int(u) < MAX_ID and int(w) < MAX_ID):
        train_set.add(line)
        nodes.add(int(u))
        nodes.add(int(w))
fin_train.close()

In [2]:
from graph_tool.all import *

In [12]:
g = Graph(directed=False)

In [13]:
g.add_vertex(max(nodes) + 1)

<generator object <genexpr> at 0x7f3024c46cd0>

In [14]:
for edge in train_set:
    u, w = map(int, edge.split())
    g.add_edge(g.vertex(u), g.vertex(w))

In [23]:
pos = sfdp_layout(g, max_iter=20, epsilon=0.5, verbose=True)
pickle.dump(pos, open( "pos-2.p", "wb" ) )

Coarse level (EC): 2  num vertices: 332144
Coarse level (EC): 3  num vertices: 219241
Coarse level (EC): 4  num vertices: 158588
Coarse level (MIVS): 5  num vertices: 123307
Coarse level (MIVS): 6  num vertices: 86950
Coarse level (MIVS): 7  num vertices: 73530
Positioning level: 0 73530 with K = 141.259468162 ...
avg edge distance: 348.816558962
propagating... 73530
Positioning level: 1 86950 with K = 105.944601122 ...
avg edge distance: 366.156833465
propagating... 86949
Positioning level: 2 123307 with K = 79.4584508412 ...
avg edge distance: 383.605146423
propagating... 
Positioning level: 3 158588 with K = 59.5938381309 ...
avg edge distance: 335.030604055
propagating... 
Positioning level: 4 219241 with K = 44.6953785982 ...
avg edge distance: 276.950184883
propagating... 
Positioning level: 5 332144 with K = 33.5215339486 ...
avg edge distance: 217.433864472
propagating... 
Positioning level: 6 546295 with K = 25.1411504615 ...
avg edge distance: 156.83436562


In [15]:
%matplotlib inline

graph_draw(g, pos=pos)

AttributeError: 'NoneType' object has no attribute 'new_vertex_property'

# Save graph

In [ ]:
g.vertex_properties["pos"] = pos
g_3.save("my_graph_newname.xml.gz")

# Load graph

In [ ]:
g = load_graph("my_graph_newname.xml.gz")
pos = g.vertex_properties["pos"]

# Assign new graph copy to pos

In [7]:
import pickle
pos = pickle.load( open( "pos.p", "rb" ) )

In [3]:
g_3 = load_graph("my_graph.xml.gz")

In [23]:
pos_g = g_3.new_vertex_property("vector<double>")

In [25]:
for v in g_3.vertices():
    pos_g[v] = pos[v]

In [29]:
g_3.vertex_properties["pos"] = pos_g

# Read pos and neg

In [40]:
fin_poss = open('poss_set.txt', 'r')
fin_neg = open('neg_set.txt', 'r')

poss_set = set()
neg_set = set()

for line in fin_poss:
    line = line.strip()
    u, w = line.split()
    if MAX_ID == -1 or (int(u) < MAX_ID and int(w) < MAX_ID):
        poss_set.add(line)
fin_poss.close()

for line in fin_neg:
    line = line.strip()
    u, w = line.split()
    if MAX_ID == -1 or (int(u) < MAX_ID and int(w) < MAX_ID):
        neg_set.add(line)
fin_neg.close()

In [41]:
print len(poss_set), len(neg_set)

100000 100000


# Предсказание

In [42]:
import numpy    
def dist(u, w):
    #if comp[g.vertex(u)] != comp[g.vertex(w)]:
    #    return 100000000
    
    u = pos[g.vertex(u)]
    w = pos[g.vertex(w)]
    return ((u[0] - w[0])**2 + (u[1] - w[1])**2)**0.5

def preferential_attachment(u, w):
    return g.vertex(u).out_degree()*g.vertex(w).out_degree()

def common_neighbors(u, w):
    return len(set.intersection(
        set(g.vertex(u).out_neighbours()), 
        set(g.vertex(w).out_neighbours())))

def union_neighbors(u, w):
    return len(
        set(g.vertex(u).out_neighbours()) | set(g.vertex(w).out_neighbours()))
            
def Jaccards_coefficient(u, w):
    return 1.0 * common_neighbors(u, w) / union_neighbors(u, w)

In [43]:
def make_dataset(poss_set, neg_set, functs):
    X = []
    Y = []
    for line in poss_set:
        u, w = map(int, line.split())
        x = []
        for func in functs:
            x.append(func(u, w))
        X.append(x)
        Y.append(1)
    for line in neg_set:
        u, w = map(int, line.split())
        x = []
        for func in functs:
            x.append(func(u, w))
        X.append(x)
        Y.append(0)
    X = numpy.array(X)
    Y = numpy.array(Y)
    return X, Y

In [44]:
X, Y = make_dataset(poss_set, neg_set, 
                    [dist
                    ])

In [45]:
X[:5], Y[:5]

(array([[ 758.71192528],
        [ 636.17866018],
        [ 192.82777073],
        [ 143.45031731],
        [ 122.6144993 ]]), array([1, 1, 1, 1, 1]))

In [46]:
X.shape, Y.shape

((200000, 1), (200000,))

In [47]:
from sklearn.metrics import roc_auc_score
roc_auc_score(Y, 1 - X)

0.83532481390000002

In [24]:
from  sklearn.ensemble import RandomForestClassifier as RF

In [25]:
from sklearn.model_selection import cross_val_score

In [26]:
clf = RF()

In [27]:
scores = cross_val_score(clf, X, Y, cv=5, scoring='roc_auc')

In [28]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.52 (+/- 0.03)


# Предсказание через MF

In [48]:
from scipy.sparse import coo_matrix
def make_sparse_matrix(train_set, n):
    row = []
    col = []
    data = []
    for line in train_set:
        u, w = map(int, line.split())
        row.append(u)
        col.append(w)
        row.append(w)
        col.append(u)
        data.append(1)
        data.append(1)
    return coo_matrix((data, (row, col)), shape=(n, n))

In [51]:
G = make_sparse_matrix(train_set, max(nodes) + 1)

In [57]:
max(nodes) + 1

546295

In [52]:
from sklearn.decomposition import NMF

In [53]:
model = NMF(n_components=10, init='random', random_state=0)

In [54]:
A = model.fit_transform(G)

In [55]:
A.shape

(546295, 10)

In [58]:
def mf_score(u, w):
    return numpy.dot(A[u], A[w])

In [59]:
X, Y = make_dataset(poss_set, neg_set, [mf_score])

In [60]:
from sklearn.metrics import roc_auc_score
roc_auc_score(Y, X.reshape((X.shape[0],)))

0.80534146170000009

In [38]:
scores = cross_val_score(clf, X, Y, cv=5, scoring='roc_auc')
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.58 (+/- 0.06)
